# Open The Cameras

## Cameras On The Base

In [1]:
from FlexivPy.vision import RealSenseCamera

In [ ]:
# Camera to the left of the robot
camera1 = RealSenseCamera(VGA=False, camera_serial_no='234222302193')

In [ ]:
# Camera to the right of the robot
camera2 = RealSenseCamera(VGA=False, camera_serial_no='231622302407')

In [ ]:
camera1.close()
camera2.close()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,2)
ax[0].imshow(camera1.color_frame)
ax[1].imshow(camera2.color_frame)

In [13]:
left_cam_ints = camera1.getIntrinsics()
right_cam_ints = camera2.getIntrinsics()

In [14]:
import cv2
def camera_callback(color_frame, depth_frame, ir1_frame, ir2_frame):
    cv2.imshow('frame', color_frame)
    cv2.waitKey(1)

In [ ]:
from FlexivPy.vision import RealSenseCamera
ef_camera = RealSenseCamera(VGA=False, camera_serial_no='141222073965', callback_fn=camera_callback)
ef_camera_ints = ef_camera.getIntrinsics()

In [ ]:
plt.imshow(ef_camera.depth_frame)

In [ ]:
from FlexivPy.joy import XBoxController
joy = XBoxController(0)

In [ ]:
joy.getStates()

In [ ]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
model = FlexivModel()
robot = FlexivDDSClient()

In [ ]:
from FlexivPy.controllers.runners import NonBlockingRunner
from FlexivPy.controllers.taskspace import DiffIKController
task_controller = DiffIKController(model, dt=0.01, dq_max=1., control_mode='velocity', max_error=0.1)

In [ ]:
runner = NonBlockingRunner(robot, task_controller, timeout=5000.)

In [ ]:
import time
import numpy as np

ef_rgbs = []
left_rgbs = []
right_rgbs = []

ef_depths=[]
left_depths=[]
right_depths=[]
link7_poses = []

time.sleep(0.2)
state = robot.get_robot_state()
old_A_state = 0
if state is not None:
    print('starting the controller')
    info = model.getInfo(np.array(state.q), np.array(state.dq))
    T0 = info['poses']['link7']
    # Initialize the desired pose
    x0, y0, z0 = 0.0, 0.0, 0.0 
    R0 = np.eye(3)
    start_time = time.time()
    while time.time()-start_time < 1000:
        joy_state = joy.getStates()
        left_joy = joy_state['left_joy']
        right_joy = joy_state['right_joy']

        if joy_state['right_bumper']==0:
            y0 = y0 + right_joy[0]*0.2/100
            x0 = x0 + right_joy[1]*0.2/100
            z0 = z0 - left_joy[1]*0.2/100
        else:
            cmd = np.array([right_joy[0],right_joy[1],left_joy[0]])
            omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                                [ cmd[2],   0,      -cmd[0]],
                                [-cmd[1],   cmd[0],      0]])
            R0 = R0@(np.eye(3)+omega_hat/100)

        time.sleep(0.01)
        T_cmd= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
        task_controller.T_cmd = T_cmd
        state = robot.get_robot_state()
        info = model.getInfo(np.array(state.q), np.array(state.dq))
        T = info['poses']['link7']
        if joy_state['A'] and old_A_state==0:
            old_A_state = joy_state['A']
            link7_poses.append(T.copy())
            left_rgbs.append(camera1.color_frame.copy())
            right_rgbs.append(camera2.color_frame.copy())
            ef_rgbs.append(ef_camera.color_frame.copy())
            left_depths.append(camera1.depth_frame.copy())
            right_depths.append(camera2.depth_frame.copy())
            ef_depths.append(ef_camera.depth_frame.copy())
            print(f'took a pictures {time.time()}')
            time.sleep(0.1)
        else:
            old_A_state=joy_state['A']

    print('Demo ended.')
else:
    print('State is None. check the connection')

In [ ]:
len(left_rgbs)

In [ ]:
import matplotlib.pyplot as plt
i=0
fig, ax = plt.subplots(1,3)
ax[0].imshow(left_rgbs[i])
ax[1].imshow(right_rgbs[i])
ax[2].imshow(ef_rgbs[i])

In [30]:
import pickle 
with open('data/3dgs_dataset1.pkl', 'wb') as f:
    pickle.dump(
        {
            'link7_poses':link7_poses,
            'left_rgbs':left_rgbs,
            'right_rgbs':right_rgbs,
            'ef_rgbs':ef_rgbs,
            'left_depths':left_depths,
            'right_depths':right_depths,
            'ef_depths':ef_depths,
            'ef_camera_ints':ef_camera_ints,
            'left_cam_ints':left_cam_ints,
            'right_cam_ints':right_cam_ints
        },
        f
    )

In [32]:
runner.close()

In [33]:
robot.close()

In [ ]:
ef_camera.close()
camera1.close()
camera2.close()